In [1]:
import numpy as np
np.random.seed(42)

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import recall_score, roc_auc_score, precision_score, matthews_corrcoef, classification_report, roc_curve
%matplotlib inline

In [3]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [4]:
df = pd.read_csv('train.csv')

In [5]:
df.head(3)

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...


In [6]:
len(df)

38740

In [7]:
df['text'] = df.groupby(['oid'])['text'].transform(lambda x: ' '.join(x))

In [8]:
df = df.drop_duplicates()

In [9]:
df['category'].value_counts()

autosport       316
extreme         311
martial_arts    305
motosport       303
boardgames      302
tennis          300
esport          299
athletics       297
volleyball      295
hockey          295
football        286
basketball      285
winter_sport    280
Name: category, dtype: int64

In [10]:
categories = [
    'athletics',
    'autosport',
    'basketball',
    'boardgames',
    'esport',
    'extreme',
    'football',
    'hockey',
    'martial_arts',
    'motosport',
    'tennis',
    'volleyball',
    'winter_sport',
]

In [11]:
cat_map = dict({
    (c, i) for i, c in enumerate(categories)
})

In [12]:
cat_map_inv = {v: k for k, v in cat_map.items()}

In [13]:
df['label'] = df['category'].apply(lambda x: cat_map[x])

In [14]:
df = df[['text', 'label']]

In [15]:
val_size = 0.3

In [16]:
X_text = df['text']
y = df['label']

In [17]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /Users/timur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/timur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
stopWords = set(stopwords.words('russian'))

In [19]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [20]:
text_sample = X_text.values[0]

In [21]:
text_sample

'Волшебные фото Виктория Поплавская ЕвгенияМедведева Evgentokenoid Саша Трусова о Жене Моховая 9 ЕвгенияМедведева Evgentokenoid 24 апреля 18 55 Первый канал ️ ЕвгенияМедведева Evgentokenoid По заявкам публики Евгения Медведева и Торнике Квитатиани 33 33 33 Шоу Вызов на ТНТ 33 33 Реальный Губер ЕвгенияМедведева Evgentokenoid Кружочки с шоу Влюбленные в фигурное катание из телеграма Жени ЕвгенияМедведева Evgentokenoid Женя на хоккее ЕвгенияМедведева Evgentokenoid Betokenoid Сегодня Женя комментировала короткую программу мужчин ️ Денис Гладков ЕвгенияМедведева Evgentokenoid Последний совместный номер Жени Медведевой и Дани Милохина ftokenoid ЕвгенияМедведева Evgentokenoid Женя рассказывает о первом дне в Карелии ️ YAPPY ЕвгенияМедведева Evgentokenoid Еще немного видео с ребятами А теперь внимание 33 ЕвгенияМедведева Evgentokenoid ЛедниковыйПериод'

In [22]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
pymorphy_results = list(map(lambda x: morph.parse(x), text_sample.split(' ')))
print(' '.join([res[0].normal_form for res in pymorphy_results]))

волшебный фото виктория поплавский евгениямедведев evgentokenoid саша трусов о жена моховой 9 евгениямедведев evgentokenoid 24 апрель 18 55 первый канал ️ евгениямедведев evgentokenoid по заявка публика евгений медведев и торник квитатиани 33 33 33 шоу вызов на тнт 33 33 реальный губер евгениямедведев evgentokenoid кружочек с шоу влюбить в фигурный катание из телеграм женить евгениямедведев evgentokenoid женя на хоккей евгениямедведев evgentokenoid betokenoid сегодня женя комментировать короткий программа мужчина ️ денис гладков евгениямедведев evgentokenoid последний совместный номер женить медведев и дань милохина ftokenoid евгениямедведев evgentokenoid женя рассказывать о первый день в карелия ️ yappy евгениямедведев evgentokenoid ещё немного видео с ребята а теперь внимание 33 евгениямедведев evgentokenoid ледниковыйпериод


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
# # lowercase
# vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=0.03, max_df=0.9)
# vectors = vectorizer.fit_transform(X)
# vectors.shape

In [25]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
pymorphy_results = list(map(lambda x: morph.parse(x), text_sample.split(' ')))
' '.join([res[0].normal_form for res in pymorphy_results])

'волшебный фото виктория поплавский евгениямедведев evgentokenoid саша трусов о жена моховой 9 евгениямедведев evgentokenoid 24 апрель 18 55 первый канал ️ евгениямедведев evgentokenoid по заявка публика евгений медведев и торник квитатиани 33 33 33 шоу вызов на тнт 33 33 реальный губер евгениямедведев evgentokenoid кружочек с шоу влюбить в фигурный катание из телеграм женить евгениямедведев evgentokenoid женя на хоккей евгениямедведев evgentokenoid betokenoid сегодня женя комментировать короткий программа мужчина ️ денис гладков евгениямедведев evgentokenoid последний совместный номер женить медведев и дань милохина ftokenoid евгениямедведев evgentokenoid женя рассказывать о первый день в карелия ️ yappy евгениямедведев evgentokenoid ещё немного видео с ребята а теперь внимание 33 евгениямедведев evgentokenoid ледниковыйпериод'

In [26]:
# # стоп-слова, preproc
# from nltk.corpus import stopwords
# stopWords = set(stopwords.words('russian'))

# def preproc_text(text):
#     pymorphy_results = list(map(lambda x: morph.parse(x), text_sample.split(' ')))
#     return ' '.join([res[0].normal_form for res in pymorphy_results])

In [92]:
wnl.stem('продам')

'прод'

In [93]:
from nltk.corpus import stopwords
stopWords = set(stopwords.words('russian'))
nltk.download('wordnet')
wnl = nltk.SnowballStemmer('russian')

[nltk_data] Downloading package wordnet to /Users/timur/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [94]:
text_sample

'Волшебные фото Виктория Поплавская ЕвгенияМедведева Evgentokenoid Саша Трусова о Жене Моховая 9 ЕвгенияМедведева Evgentokenoid 24 апреля 18 55 Первый канал ️ ЕвгенияМедведева Evgentokenoid По заявкам публики Евгения Медведева и Торнике Квитатиани 33 33 33 Шоу Вызов на ТНТ 33 33 Реальный Губер ЕвгенияМедведева Evgentokenoid Кружочки с шоу Влюбленные в фигурное катание из телеграма Жени ЕвгенияМедведева Evgentokenoid Женя на хоккее ЕвгенияМедведева Evgentokenoid Betokenoid Сегодня Женя комментировала короткую программу мужчин ️ Денис Гладков ЕвгенияМедведева Evgentokenoid Последний совместный номер Жени Медведевой и Дани Милохина ftokenoid ЕвгенияМедведева Evgentokenoid Женя рассказывает о первом дне в Карелии ️ YAPPY ЕвгенияМедведева Evgentokenoid Еще немного видео с ребятами А теперь внимание 33 ЕвгенияМедведева Evgentokenoid ЛедниковыйПериод'

In [97]:
stopWords = stopWords | {'продать', 'реклама', 'подробней', 'ссылке'}

In [98]:
def preproc_nltk(text):
    #text = re.sub(f'[{string.punctuation}]', ' ', text)
#     return ' '.join([wnl.lemmatize(word) for word in word_tokenize(text.lower()) if word not in stopWords])
    return ' '.join([wnl.stem(word) for word in word_tokenize(text.lower()) if word not in stopWords])

In [105]:
vectorizer = TfidfVectorizer(preprocessor=preproc_nltk)

In [106]:
vectors = vectorizer.fit_transform(X_text)

In [107]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import SGDClassifier 

In [108]:
X = np.asarray(vectors.todense())

In [109]:
val_size = 0.3
X_train, X_val, y_train, y_val= train_test_split(X, y, stratify=y, test_size=val_size, random_state=42)
# X_train, X_val, y_train, y_val = np.asarray(X_train), np.asarray(X_val), np.asarray(y_train), np.asarray(y_val)

In [110]:
X_train.shape

(2711, 70018)

In [111]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [112]:
clf = SGDClassifier(max_iter=1000, warm_start=True, tol=1e-8)

In [113]:
# clf = SVC(verbose=2)

In [114]:
clf.fit(X_train, y_train)

SGDClassifier(tol=1e-08, warm_start=True)

In [115]:
val_pred = clf.predict(X_val)

In [116]:
accuracy_score(y_val, val_pred)

0.9819432502149613

In [117]:
# accuracy_score(y_val, val_pred)
# 0.9836629406706793

In [118]:
clf.fit(X, y)

SGDClassifier(tol=1e-08, warm_start=True)

In [119]:
test = pd.read_csv('test.csv')

In [120]:
test.head(3)

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...


In [133]:
# prev = pd.read_csv('solution.csv')

In [139]:
# prev.loc[0]['category'] = 'tennis'/

In [141]:
# prev.to_csv('solution_zh.csv', index=False)

In [121]:
test['text'] = test.groupby(['oid'])['text'].transform(lambda x: ' '.join(x))

In [122]:
test = test.drop_duplicates()

In [123]:
test_vectors = vectorizer.transform(test['text'])

In [124]:
test_pred = clf.predict(test_vectors)

In [125]:
# test_pred_proba = np.max(clf.predict_proba(test_vectors), 1)

In [126]:
# test['proba'] = test_pred_proba

In [127]:
test['label'] = test_pred

In [128]:
solution = pd.read_csv('sample_submission.csv')

In [129]:
test['category'] = test['label'].apply(lambda x: cat_map_inv[x])

In [130]:
# solution = test[['oid', 'category', 'proba']]

In [131]:
solution = test[['oid', 'category']]

In [132]:
solution.to_csv('solution2.csv', index=False)

In [62]:
for t in test[solution.proba < 0.2]['text'].values:
    print(t)
    print('---------')

Марчеллис сеть семейных ресторанов итальянской кухни в Санкт Петербурге и Москве. Подробней по ссылке Кисляк Вы хотите сказать что мы не старались? Макеев Нет не хочу но ваши старания были похожи на басню Крылова Лебедь рак и щука. Егор извини за каламбур. Романтичная и нежная Анна Уколова на съемках нового сезона сериала Ивановы Ивановы. Андрей Привет девчонки 33 Миша Здорова Кислый 33 Жданов Я не понял ты где здесь девчонок то увидел? Может я девчонка? Или Бушманов? Или допустим Юдин тоже девчонка? Андрей Да нет это я так. Жданов Че так? Со зрением проблемы? Андрей Нет. Просто по привычке. Жданов Привычки надо менять не в молодежке уже. Надо думать с кем и как разговариваешь понял? Как менялась Мария Пирогова на протяжении 5 сезонов сериала Молодежка. По премьеры нового сезона Дылды осталось 7 дней. Будите смотреть? Для чего нужен студенческий? Показать на входе в университет доказать бабушке что ты реально поступил а самое главное – оформить Плюс Мульти бесплатно до конца года 33 Ка

In [110]:
THRESH = 0.32

In [111]:
final_solution = solution[solution.proba > THRESH][['oid', 'category']]

In [112]:
len(final_solution)

2552